# Julia Multi-Threading 

[![ ](https://markdown-videos-api.jorgenkh.no/url?url=https%3A%2F%2Fyoutu.be%2FkX6_iY_BtG8%3Fsi%3DQELqXGFHALzGIeGR)](https://youtu.be/kX6_iY_BtG8?si=QELqXGFHALzGIeGR)

### 👉 number of threads 

In [ ]:
versioninfo()

In [ ]:
Threads.nthreads()

### 👉 creating a multi-thread kernel for jupyter

In [ ]:
using IJulia

In [ ]:
# Creating a new kernel where "Julia Multi-Thread" is the name 
# and "4" is the number of threads.
# you can change the name and number of threads

installkernel("Julia Multi-Thread", env=Dict("JULIA_NUM_THREADS"=>"4"))

#### 💡 Refresh the page and change the kernel (Julia Multi-Thread)

### 👉 single-thread vs multi-thread

In [ ]:
# number of threads

N = Threads.nthreads()

In [ ]:
# single-threaded

for i in 1:N
    println("i: ", i, "\t Thread ID: ", Threads.threadid())
end

In [ ]:
# multi-threaded

Threads.@threads for i in 1:N
    println("i: ", i, "\t Thread ID: ", Threads.threadid())
end

### 👉 data-race example

In [ ]:
# creating a vector and calculating the sum

n = 1_000_000
myvector = collect(1:n)
sum(myvector)

In [ ]:
# single-threaded

function multi_sum1(myvector)
    temp = 0

    for i in eachindex(myvector)
        temp += myvector[i]
    end

    return temp
end

In [ ]:
multi_sum1(myvector)

In [ ]:
# multi-threaded

function multi_sum2(myvector)
    temp = 0

    Threads.@threads for i in eachindex(myvector)
        temp += myvector[i]
    end

    return temp
end

In [ ]:
# re-run this cell several times and observe the output

multi_sum2(myvector)

🚩🚩🚩 The multi_sum2() function gives wrong values 

In [ ]:
# multi-threaded without data-race test function

function multi_sum3test(myvector)
    temp = zeros(Int, Threads.nthreads())

    Threads.@threads for i in eachindex(myvector)
        temp[Threads.threadid()] += myvector[i]
    end

    for i in eachindex(temp)
        println(i, "\t = ", temp[i])
    end

    println()
    println("The sum is ", sum(temp))
end

In [ ]:
multi_sum3test(myvector)

In [ ]:
# multi-threaded without data-race

function multi_sum3(myvector)
    temp = zeros(Int, Threads.nthreads())

    Threads.@threads for i in eachindex(myvector)
        temp[Threads.threadid()] += myvector[i]
    end

    return sum(temp)
end

In [ ]:
multi_sum3(myvector)

### speed test

In [ ]:
# Re-run the cell several times

@time multi_sum1(myvector);

In [ ]:
# Re-run the cell several times

@time multi_sum3(myvector);

In [ ]:
# Re-run the cell several times

@time sum(myvector);

### speed test with a larger vector

In [ ]:
# trying with a larger vector

n = 100_000_000
myvector = collect(1:n);

In [ ]:
# Re-run the cell several times

@time multi_sum3(myvector);

In [ ]:
# Re-run the cell several times

@time sum(myvector);

### 👉 multi-threaded axpy()

use multi-threading, 32-bit, in-place

In [ ]:
# 32-bit constants

const m::Int32 = 100_000
const a::Float32 = 2.718

In [ ]:
# define multi-threaded axpy() function

function multi_axpy!(a, x, y)
    Threads.@threads for i in eachindex(x)
        y[i] = a*x[i] + y[i]
        # @inbounds y[i] = a*x[i] + y[i]
    end

    return y
end

In [ ]:
using Random

In [ ]:
# Re-run the cell several times

Random.seed!(1)

x = rand(Float32, m)
y = rand(Float32, m)

@time multi_axpy!(a, x, y);